In [2]:
from heatmap import to_heatmap
from heatmap import synset_to_dfs_ids
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import model_from_json

Using Theano backend.


In [3]:
def display_heatmap(new_model, img_path):

    plt.figure()
    img=mpimg.imread(img_path)
    plt.subplot(121)
    plt.imshow(img)
    
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    out = new_model.predict(x)

    s = "n02512053" # Imagenet code for "fish"
    ids = synset_to_dfs_ids(s)
    heatmap_fish = out[0,ids].sum(axis=0)
    plt.subplot(122)
    plt.imshow(heatmap_fish, interpolation="none")
    plt.show()

## Let's try with a VGG16:

In [ ]:
model = VGG19()
new_model = to_heatmap(model)

In [ ]:
display_heatmap(new_model, "../../data/train/ALB/img_00110.jpg")
display_heatmap(new_model, "../../data/train/ALB/img_00003.jpg")
display_heatmap(new_model, "../../data/train/ALB/img_00085.jpg")

## Now with a ResNet50:

In [4]:
model = ResNet50()
new_model = to_heatmap(model)

Model type detected: local pooling - flatten


Exception: You are attempting to share a same `BatchNormalization` layer across different data flows. This is not possible. You should use `mode=2` in `BatchNormalization`, which has a similar behavior but is shareable (see docs for a description of the behavior).

In [ ]:
display_heatmap(new_model, "../../data/train/LAG/img_00176.jpg")
display_heatmap(new_model, "../../data/train/ALB/img_00003.jpg")
display_heatmap(new_model, "../../data/train/SHARK/img_00072.jpg")

## Now with a custom classifier:

Class 0 is "fish" and class 1 is "no fish"

In [ ]:
# load json and create model
json_file = open('model_2c_10e_R50_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model_2c_10e_R50_1.h5")
print("Loaded model from disk")

In [ ]:
new_model = to_heatmap(model, input_shape=(3,256,256))

In [ ]:
def display_heatmap(new_model, img_path):

    plt.figure()
    img=mpimg.imread(img_path)
    plt.subplot(121)
    plt.imshow(img)
    
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    out = new_model.predict(x)

    heatmap_fish = out[0,[0]].sum(axis=0)
    plt.subplot(122)
    plt.imshow(heatmap_fish, interpolation="none")
    plt.show()

In [ ]:
display_heatmap(new_model, "../../data/train/LAG/img_00176.jpg")
display_heatmap(new_model, "../../data/train/ALB/img_00003.jpg")
display_heatmap(new_model, "../../data/train/SHARK/img_00072.jpg")

## Now with the InceptionV3:

It's buggy and I don't know why. If someone could figure it out, it'd be great.

In [ ]:
model = InceptionV3()
new_model = to_heatmap(model)

In [ ]:
import cv2
print cv2.__version__

def display_heatmap(new_model, img_path):

    plt.figure()
    img=cv2.imread(img_path)
    ret,thresh = cv2.threshold(img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    #cnt = contours[0]
    #M = cv2.moments(cnt)
    #print M
    #rect = cv2.minAreaRect(cnt)
    #box = cv2.boxPoints(rect)
    #box = np.int0(box)
    #cv2.drawContours(img,[box],0,(0,0,255),2)
    plt.subplot(121)
    plt.imshow(img)
    
    img = image.load_img(img_path)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    out = new_model.predict(x)

    s = "n02512053" # Imagenet code for "fish"
    ids = synset_to_dfs_ids(s)
    heatmap_fish = out[0,ids].sum(axis=0)
    plt.subplot(122)
    plt.imshow(heatmap_fish, interpolation="none")
    plt.show()

In [ ]:
display_heatmap(new_model, "../../data/train/LAG/img_00176.jpg")
display_heatmap(new_model, "../../data/train/ALB/img_00003.jpg")
display_heatmap(new_model, "../../data/train/SHARK/img_00072.jpg")

Bounding boxes

In [ ]:
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(img,[box],0,(0,0,255),2)

Don't hesitate to contribute!